In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# changed and escaped slashes in path, as i'm on windows
file_dir = "C:\\Users\\me\\Desktop\\0 berbooca\\github\\Movies-ETL\\"

### wikimedia

In [3]:
raw_file = f"{file_dir}wikipedia-movies.json"

In [4]:
with open(raw_file, mode='r') as file:
    wiki_movies_raw = json.load(file)

In [7]:
len(wiki_movies_raw)

7311

In [8]:
type(wiki_movies_raw)

list

In [10]:
wiki_movies_raw[2288]

{'url': 'https://en.wikipedia.org/wiki/Enemy_of_the_State_(film)',
 'year': 1998,
 'imdb_link': 'https://www.imdb.com/title/tt0120660/',
 'title': 'Enemy of the State',
 'Directed by': 'Tony Scott',
 'Produced by': 'Jerry Bruckheimer',
 'Written by': 'David Marconi',
 'Starring': ['Will Smith',
  'Gene Hackman',
  'Jon Voight',
  'Regina King',
  'Loren Dean',
  'Jack Black',
  'Seth Green',
  'Jake Busey',
  'Scott Caan',
  'Barry Pepper',
  'Jamie Kennedy',
  'Gabriel Byrne'],
 'Music by': ['Harry Gregson-Williams', '[1]', 'Trevor Rabin'],
 'Cinematography': 'Dan Mindel',
 'Edited by': 'Chris Lebenzon',
 'Productioncompany ': ['Touchstone Pictures',
  'Jerry Bruckheimer Films',
  'Scott Free Productions'],
 'Distributed by': 'Buena Vista Pictures',
 'Release date': ['November 20, 1998', '(', '1998-11-20', ')'],
 'Running time': '132 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$90 million',
 'Box office': '$250.6 million'}

In [17]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [20]:
col_list = wiki_movies_df.columns.tolist()

In [21]:
len(col_list)

193

In [22]:
wiki_movies_df["imdb_link"].count()
# i think this means 7100 out of 7311 rows have imdb links; i assume the rest are NaN

7100

In [23]:
wiki_movies_df["Director"].count(), wiki_movies_df["Directed by"].count()
# i think this says only 2 rows have a "Director" entry, but 7103 have "Directed by" entry

(2, 7103)

In [24]:
noimdb_df = wiki_movies_df[wiki_movies_df["imdb_link"].isna()]

In [25]:
len(noimdb_df)

211

In [28]:
noimdb_df["Director"].count()
# 25 have "Directed by", 0 have "Director"

0

So, removing ones without director/directed-by, and without imdb link, will only drop ~250 records.

In [29]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [30]:
len(wiki_movies)
# lost 311+24 = 335

7076

In [31]:
wiki_movies_df = pd.DataFrame(wiki_movies)

In [32]:
wiki_movies_df.columns

Index(['url', 'year', 'imdb_link', 'title', 'Directed by', 'Produced by',
       'Screenplay by', 'Story by', 'Based on', 'Starring', 'Narrated by',
       'Music by', 'Cinematography', 'Edited by', 'Productioncompany ',
       'Distributed by', 'Release date', 'Running time', 'Country', 'Language',
       'Budget', 'Box office', 'Written by', 'Genre', 'Theme music composer',
       'Country of origin', 'Original language(s)', 'Producer(s)', 'Editor(s)',
       'Production company(s)', 'Original network', 'Original release',
       'Productioncompanies ', 'Executive producer(s)',
       'Production location(s)', 'Distributor', 'Picture format',
       'Audio format', 'Voices of', 'Followed by', 'Composer(s)', 'Created by',
       'Preceded by', 'Adaptation by', 'Suggested by', 'Traditional',
       'Mandarin', 'Released', 'Recorded', 'Venue', 'Length', 'Label',
       'Director', 'Producer', 'Also known as', 'Animation by',
       'Color process', 'Simplified', 'French', 'Screen story 

In [40]:
wiki_movies_df["Starring"]
# beware, beware: these are passed-by-reference, even with .copy() !!

0       [Andrew Dice Clay, Wayne Newton, Priscilla Pre...
1       [Jason Patric, Rachel Ward, Bruce Dern, George...
2       [Mel Gibson, Robert Downey Jr., Nancy Travis, ...
3       [Alec Baldwin, Blythe Danner, Judy Davis, Mia ...
4             [Paul Hogan, Elias Koteas, Linda Kozlowski]
                              ...                        
7071    [Will Ferrell, John C. Reilly, Rebecca Hall, R...
7072    [Christian Bale, Amy Adams, Steve Carell, Sam ...
7073    [Felicity Jones, Armie Hammer, Justin Theroux,...
7074    [Nicole Kidman, Sebastian Stan, Toby Kebbell, ...
7075    [Fionn Whitehead, Will Poulter, Craig Parkinso...
Name: Starring, Length: 7076, dtype: object

### kaggle

In [5]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [6]:
ratings.sample(3)

,userId,movieId,rating,timestamp
9213426,95117,616,3.0,835801027
10751505,111002,2918,1.0,1138705003
18854189,195800,4993,5.0,1482157574


In [7]:
kaggle_metadata.sample(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
29349,False,NaN,3673315,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",http://www.alleluia-lefilm.com/,269776,tt3218580,fr,Alléluia,"Michel, a murderous womanizer, meets introvert...",...,2014-05-14,0.0,93.0,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Released,Fervidly dark lovesick horror inspired by the ...,Alléluia,False,6.3,24.0
33065,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 27, 'n...",NaN,353686,tt2954474,en,Visions,"After moving to a vineyard with her family, a ...",...,2015-08-28,0.0,80.0,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Released,Blood will stain the land.,Visions,False,5.2,130.0
27631,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",NaN,222870,tt0116319,en,Subliminal Seduction,Darrin Danver is married to Deb. He is a video...,...,1996-01-01,0.0,81.0,[],Released,NaN,Subliminal Seduction,False,4.0,3.0


In [10]:
kaggle_metadata.count()

adult                    45466
belongs_to_collection     4494
budget                   45466
genres                   45466
homepage                  7782
id                       45466
imdb_id                  45449
original_language        45455
original_title           45466
overview                 44512
popularity               45461
poster_path              45080
production_companies     45463
production_countries     45463
release_date             45379
revenue                  45460
runtime                  45203
spoken_languages         45460
status                   45379
tagline                  20412
title                    45460
video                    45460
vote_average             45460
vote_count               45460
dtype: int64

In [12]:
kaggle_metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [16]:
kaggle_metadata[kaggle_metadata["vote_count"].isna()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19729,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,82663,tt0113002,en,Midnight Man,British soldiers force a recently captured IRA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29502,False,"{'id': 122661, 'name': 'Mardock Scramble Colle...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",http://m-scramble.jp/exhaust/,122662,tt2423504,ja,マルドゥック・スクランブル 排気,Third film of the Mardock Scramble series.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35586,False,NaN,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...",NaN,249260,tt2622826,en,Avalanche Sharks,A group of skiers are terrorized during spring...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
